In [1]:
import os

In [2]:
%pwd

'/home/vishal/Vishal/Chicken_Disease_Detection/research'

In [3]:
os.chdir("../")

In [56]:
%pwd

'/home/vishal/Vishal/Chicken_Disease_Detection'

In [57]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [58]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [59]:
from cnnClassifier import logger

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)  # Should be a ConfigBox
        self.params = read_yaml(params_filepath)  # Should also be a ConfigBox
        
        # Ensure that artifacts_root exists
        if 'artifacts_root' not in self.config:
            logger.error("The configuration does not contain 'artifacts_root'.")
            raise KeyError("Missing 'artifacts_root' in the configuration file.")
        
        logger.info(f"artifacts_root found: {self.config.artifacts_root}")
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [60]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [61]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [62]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

EnsureError: Return value of <function read_yaml at 0x74015d517060> of type <class 'dict'> does not match annotation type <class 'box.config_box.ConfigBox'>